In [ ]:
import geopandas as gp
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import pearsonr, spearmanr
import networkx as nx
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from haversine import haversine
import warnings
from tqdm.notebook import tqdm

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
files = []

for i in range(2020,2023):
    for j in range(12):
        files.append("traffic_density_{}".format(str(i) + str(j+1).zfill(2)))

for i in range(4):
    files.append("traffic_density_{}".format("2023" + str(i+1).zfill(2)))

In [ ]:
for file in files:
    df = pd.read_csv("~/Downloads/Graph Files/" + file + "_corr_distance.csv")

    df["Spearman Correlation"] = abs(df["Spearman Correlation"])

    df = df[df["Distance"] <= 100]
    
    df["Bin"] = pd.cut(df['Distance'], bins = 21, precision = 6)

    distances = df.groupby("Bin")["Spearman Correlation"].agg([np.mean, np.std]).reset_index()

    bins = [round(np.mean([dist.right, dist.left]), 2) for dist in distances["Bin"]]
    
    coeff = np.polyfit([bins[i] for i in range(0, 21, 5)], [distances["mean"][i] for i in range(0, 21, 5)], deg=1)
    
    eq = np.poly1d(coeff)

    plt.figure(figsize = (12, 6))

    plt.plot(bins, eq(bins), c = "red", zorder = 1)
    plt.errorbar(bins, distances["mean"], yerr = distances["std"], marker = "o", markersize = 10, zorder = -1)
    plt.plot()
    plt.ylim([0.4, 1])
    plt.title("Distance vs. Correlation - " + file.split("_")[2])
    plt.xlabel("Distance (in km)")
    plt.ylabel("Spearman Correlation")

    plt.savefig("Correlations/" + file + "_dist_corr.png", dpi = 300)